# **Batch Speech-to-Text (STT) API Tutorial Using Saarika Model**

## **Overview**

This comprehensive guide demonstrates how to use Sarvam AI's **Batch Speech-to-Text (STT) API** for transcribing audio files at scale. You'll learn:

- **Synchronous** and **asynchronous** usage patterns
- **Webhook integration** for real-time notifications
- Key parameters and configuration options
- File upload, job management, and result download
- Best practices for production deployment

### **Key Features**

- **Processing**: Supports up to 1 hour audio files
- **Multiple Files**: Upload up to 20 audio files per job
- **Advanced Features**: Speaker diarization and timestamp support
- **Languages**: Indian languages and English with automatic detection
- **Webhook Support**: Real-time job completion notifications

### **Quick Start Checklist**

Before running this notebook:

1. ✅ **Get API Key**: Sign up at [dashboard.sarvam.ai](https://dashboard.sarvam.ai)
2. ✅ **Prepare Audio**: Have your audio files ready (MP3, WAV, etc.)
3. ✅ **Set Paths**: Update file paths in the code examples
4. ✅ **Choose Method**: Decide between sync, async, or webhook approach


## **1. Installation**

Install the Sarvam AI Python SDK:


In [ ]:
!pip install -U sarvamai


## **2. API Key Setup**

1. **Get your API key:** Sign up at the [Sarvam AI Dashboard](https://dashboard.sarvam.ai/) to obtain your API key.
2. **Set your API key:** Replace `"YOUR_API_KEY_HERE"` in the code below with your actual key.


In [ ]:
API_KEY = "YOUR_API_KEY_HERE"

## **3. Understanding STT Parameters**

### **Job Configuration Parameters**

| Parameter          | Type    | Description                          | Example                       |
| ------------------ | ------- | ------------------------------------ | ----------------------------- |
| `language_code`    | string  | Language of input audio              | `"en-IN"` (Indian English)    |
| `model`            | string  | Transcription model                  | `"saarika:v2.5"` (latest STT) |
| `with_timestamps`  | boolean | Include chunk-level timestamps       | `True`                        |
| `with_diarization` | boolean | Enable speaker identification        | `True`                        |
| `num_speakers`     | integer | Number of speakers (for diarization) | `2`                           |

### **File Upload Parameters**

| Parameter    | Type  | Description               | Default  |
| ------------ | ----- | ------------------------- | -------- |
| `file_paths` | list  | List of audio file paths  | Required |
| `timeout`    | float | Upload timeout in seconds | `60.0`   |

### **Job Monitoring Parameters**

| Parameter       | Type    | Description                      | Default |
| --------------- | ------- | -------------------------------- | ------- |
| `poll_interval` | integer | Status check frequency (seconds) | `5`     |
| `timeout`       | integer | Max wait time (seconds)          | `600`   |

### **Supported Audio Formats**

- **Formats**: MP3, WAV, FLAC, M4A, OGG
- **Duration**: Up to 1 hour per file
- **File Limit**: Up to 20 files per job
- **Quality**: Higher quality audio = better transcription accuracy


## **4. Synchronous STT Batch Example**

This approach waits for the job to complete before proceeding. Best for:

- Interactive applications
- Small to medium files
- Simple workflows


In [ ]:
from pathlib import Path
from sarvamai import SarvamAI

API_KEY = "YOUR_API_KEY_HERE"
audio_files = ["/path/to/your/audio.mp3"]  # Update with your file path
output_dir = Path("/output")
output_dir.mkdir(exist_ok=True)


def run_stt_sync():
    client = SarvamAI(api_subscription_key=API_KEY)
    job = client.speech_to_text_job.create_job(
        model="saarika:v2.5",
        with_diarization=True,
        with_timestamps=True,
        language_code="en-IN",
        num_speakers=2,
    )
    print(f"Job created: {job._job_id}")
    job.upload_files(file_paths=audio_files, timeout=120.0)
    job.start()
    print("Transcription started...")
    job.wait_until_complete(poll_interval=5, timeout=60)

    if job.is_failed():
        raise RuntimeError("Transcription failed")

    job.download_outputs(output_dir=str(output_dir))
    print(f"Transcription completed. Output saved to: {output_dir}")


run_stt_sync()

## **5. Asynchronous STT Batch Example**

This approach allows non-blocking operations. Best for:

- Large files or multiple jobs
- Server applications
- Concurrent processing


In [ ]:
import asyncio
from pathlib import Path
from sarvamai import AsyncSarvamAI

API_KEY = "YOUR_API_KEY_HERE"
audio_files = ["/path/to/your/audio.mp3"]  # Update with your file path
output_dir = Path("/output")
output_dir.mkdir(exist_ok=True)


async def run_stt_async_job():
    client = AsyncSarvamAI(api_subscription_key=API_KEY)
    job = await client.speech_to_text_job.create_job(
        model="saarika:v2.5",
        with_diarization=True,
        with_timestamps=True,
        language_code="en-IN",
        num_speakers=2,
    )
    print(f"Job created: {job._job_id}")
    await job.upload_files(file_paths=audio_files, timeout=120.0)
    await job.start()
    print("Transcription started...")
    await job.wait_until_complete(poll_interval=5, timeout=60)

    if job.is_failed():
        raise RuntimeError("Transcription failed")

    await job.download_outputs(output_dir=str(output_dir))
    print(f"Transcription completed. Output saved to: {output_dir}")


# For Jupyter environments:
import nest_asyncio

nest_asyncio.apply()
await run_stt_async_job()

## **6. Webhook Integration**

Webhooks eliminate the need for polling by sending notifications when jobs complete. This is ideal for:

- Production applications
- Long-running jobs
- Event-driven architectures

### **Webhook Benefits**

- ⚡ **Real-time notifications** - No polling required
- 🔒 **Secure** - Token-based authentication
- 📊 **Detailed payload** - Complete job information
- 🚀 **Scalable** - Handle multiple concurrent jobs

### **Webhook Server Setup**

First, let's create a simple webhook server to receive notifications:


In [ ]:
# Install FastAPI for webhook server (run this once)
!pip install fastapi uvicorn


In [ ]:
# webhook_server.py - Save this as a separate file for production use

from fastapi import FastAPI, Request, HTTPException
import uvicorn
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI(title="STT Webhook Server")

# Your webhook authentication token
VALID_TOKEN = "your-secret-webhook-token-123"


@app.post("/stt-webhook")
async def stt_webhook_handler(request: Request):
    """Handle STT job completion webhooks"""

    # Verify authentication token
    token = request.headers.get("X-SARVAM-JOB-CALLBACK-TOKEN")

    if not token or token != VALID_TOKEN:
        logger.warning(f"Invalid token received: {token}")
        raise HTTPException(status_code=403, detail="Invalid authentication token")

    # Parse webhook payload
    try:
        data = await request.json()

        job_id = data.get("job_id")
        job_state = data.get("job_state")
        created_at = data.get("created_at")
        completed_at = data.get("completed_at")
        total_files = data.get("total_files", 0)
        processed_files = data.get("processed_files", 0)
        failed_files = data.get("failed_files", 0)

        logger.info(f"📨 Webhook received for job {job_id}")
        logger.info(f"   Status: {job_state}")
        logger.info(
            f"   Files: {processed_files}/{total_files} processed, {failed_files} failed"
        )

        if job_state == "completed":
            logger.info(f"✅ Job {job_id} completed successfully!")
            # TODO: Download results, send notifications, update database, etc.
            await handle_job_completion(job_id, data)

        elif job_state == "failed":
            logger.error(f"❌ Job {job_id} failed!")
            # TODO: Handle failure, send alerts, retry logic, etc.
            await handle_job_failure(job_id, data)

        return {
            "status": "received",
            "job_id": job_id,
            "timestamp": datetime.now().isoformat(),
        }

    except Exception as e:
        logger.error(f"Error processing webhook: {e}")
        raise HTTPException(status_code=400, detail="Invalid webhook payload")


async def handle_job_completion(job_id: str, data: dict):
    """Handle successful job completion"""
    # Add your custom logic here:
    # - Download transcription results
    # - Send email notifications
    # - Update database records
    # - Trigger downstream processes
    logger.info(f"Processing completed job: {job_id}")


async def handle_job_failure(job_id: str, data: dict):
    """Handle job failure"""
    # Add your custom logic here:
    # - Log failure details
    # - Send alert notifications
    # - Implement retry logic
    # - Update monitoring systems
    logger.error(f"Handling failed job: {job_id}")


@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "timestamp": datetime.now().isoformat()}


# To run the server (uncomment for production):
# if __name__ == "__main__":
#     uvicorn.run(app, host="0.0.0.0", port=8000)

print("📋 Webhook server code ready!")
print("💡 To run: uvicorn webhook_server:app --host 0.0.0.0 --port 8000")
print("🔗 Webhook URL: http://your-domain.com:8000/stt-webhook")

### **STT Job with Webhook**

Now let's create an STT job that uses webhooks for notifications:


In [ ]:
import asyncio
import logging
from pathlib import Path
from sarvamai import AsyncSarvamAI, SarvamAIEnvironment, BulkJobCallbackParams

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration
API_KEY = "YOUR_API_KEY_HERE"
WEBHOOK_URL = "https://your-domain.com/stt-webhook"  # Your webhook endpoint
WEBHOOK_TOKEN = "your-secret-webhook-token-123"  # Your authentication token
audio_files = ["/path/to/your/audio.mp3"]  # Update with your file paths


async def run_stt_with_webhook():
    """Run STT batch job with webhook notifications"""
    print("🚀 Starting STT batch job with webhook...")

    # Initialize async client
    environment = SarvamAIEnvironment(
        base="https://api.sarvam.ai", production="wss://api.sarvam.ai"
    )
    client = AsyncSarvamAI(
        api_subscription_key=API_KEY,
        environment=environment,
    )

    # Configure webhook callback
    callback = BulkJobCallbackParams(
        url=WEBHOOK_URL,
        auth_token=WEBHOOK_TOKEN,
    )

    # Create job with webhook
    job = await client.speech_to_text_job.create_job(
        model="saarika:v2.5",
        with_diarization=True,
        with_timestamps=True,
        language_code="en-IN",
        num_speakers=2,
        callback=callback,  # Enable webhook notifications
    )

    logger.info(f"✅ Job created with ID: {job.job_id}")
    logger.info(f"🔗 Webhook configured: {WEBHOOK_URL}")

    # Upload audio files
    print("📤 Uploading audio files...")
    await job.upload_files(file_paths=audio_files, timeout=120.0)
    logger.info("✅ All files uploaded successfully")

    # Start job processing
    await job.start()
    logger.info(f"🔄 Started processing for job ID: {job.job_id}")

    # Optional: Check initial status (webhook will notify when complete)
    try:
        # Wait briefly to check if job starts successfully
        final_status = await job.wait_until_complete(timeout=30)
        logger.info(f"📊 Job status: {final_status.job_state}")

        if final_status.job_state == "completed":
            logger.info("✅ Job completed quickly! Webhook should have been triggered.")
        else:
            logger.info("⏳ Job still processing. Webhook will notify when complete.")

    except Exception as e:
        # Job is still running - webhook will handle completion
        logger.info(f"⏳ Job running in background. Webhook will notify completion.")
        logger.info(f"📋 Job ID for reference: {job.job_id}")

    return job.job_id


# For Jupyter environments:
import nest_asyncio

nest_asyncio.apply()

# Uncomment to run (make sure your webhook server is running first!)
# job_id = await run_stt_with_webhook()
# print(f"🎯 Job submitted with ID: {job_id}")
# print("📨 Check your webhook server for completion notification!")

### **Webhook Payload Structure**

When your job completes, Sarvam will send a POST request to your webhook URL with this payload:

```json
{
  "job_id": "job_12345",
  "job_state": "completed",
  "created_at": "2024-01-15T10:30:00Z",
  "completed_at": "2024-01-15T10:35:00Z",
  "total_files": 2,
  "processed_files": 2,
  "failed_files": 0
}
```

### **Webhook Security Best Practices**

- 🔒 **Always use HTTPS** for webhook URLs in production
- 🎫 **Validate auth tokens** on every request
- 📝 **Log all webhook events** for debugging
- ⚡ **Respond quickly** (< 30 seconds) to avoid timeouts
- 🔄 **Implement idempotency** to handle duplicate notifications
- 🚨 **Monitor webhook health** and implement alerting


## **7. Tips & Best Practices**

- **Audio Quality:** Use clear audio for best results.
- **Diarization:** Set `with_diarization=True` and specify `num_speakers` for multi-speaker audio.
- **Polling:** Adjust `poll_interval` and `timeout` based on expected job duration and file size.
- **Output:** Results are saved in the specified `output_dir`.
- **API Key Security:** Keep your API key confidential.
- **Webhooks:** Use webhooks for production applications to avoid polling.
- **File Formats:** Supported formats include MP3, WAV, FLAC, M4A, OGG.
- **File Limits:** Up to 20 files per job, 1 hour duration per file.


## **8. Error Handling**

You may encounter these errors while using the API:

- **403 Forbidden** (`invalid_api_key_error`)
  - Cause: Invalid API key.
  - Solution: Use a valid API key from the [Sarvam AI Dashboard](https://dashboard.sarvam.ai/).

- **429 Too Many Requests** (`insufficient_quota_error`)
  - Cause: Exceeded API quota.
  - Solution: Check your usage, upgrade if needed, or implement exponential backoff when retrying.

- **500 Internal Server Error** (`internal_server_error`)
  - Cause: Issue on our servers.
  - Solution: Try again later. If persistent, contact support.

- **400 Bad Request** (`invalid_request_error`)
  - Cause: Incorrect request formatting.
  - Solution: Verify your request structure, and parameters.

- **422 Unprocessable Entity Request** (`unprocessable_entity_error`)
  - Cause: Unable to detect the language of the input text.
  - Solution: Explicitly pass the source_language_code parameter with a supported language.


## **9. Additional Resources**

For more details, refer to the our official documentation and we are always there to support and help you on our Discord Server:

- **Documentation**: [docs.sarvam.ai](https://docs.sarvam.ai)
- **Community**: [Join the Discord Community](https://discord.com/invite/5rAsykttcs)
- **Dashboard**: [dashboard.sarvam.ai](https://dashboard.sarvam.ai)
- **GitHub**: [Sarvam AI Cookbook](https://github.com/SarvamAI/sarvam-ai-cookbook)


## **10. Final Notes**

### **🎯 Key Takeaways**

- **Choose the Right Method**: Sync for simple use cases, async for scalability, webhooks for production
- **Handle Errors Gracefully**: Implement proper error handling and retry logic
- **Secure Your Integration**: Protect API keys and validate webhook tokens
- **Optimize for Scale**: Use batching, concurrent processing, and proper resource management

### **🚀 Next Steps**

1. **Start Simple**: Begin with synchronous examples to understand the basics
2. **Add Webhooks**: Implement webhook integration for production use
3. **Monitor & Optimize**: Add logging, monitoring, and performance tracking
4. **Scale Up**: Implement concurrent processing and error recovery
5. **Go Live**: Deploy to production with proper security and monitoring

### **💡 Need Help?**

- **Technical Issues**: Check our [documentation](https://docs.sarvam.ai) or join [Discord](https://discord.com/invite/5rAsykttcs)
- **API Limits**: Contact support for quota increases
- **Custom Requirements**: Reach out for enterprise solutions at developer@sarvam.ai

---

**Keep Building!** 🚀

